# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

! wget https://www.dropbox.com/s/r0q0p0uprhcp8bb/train-sample.zip
! unzip train-sample.zip

! wget https://www.dropbox.com/s/50vw2gsglc91f6o/train.zip
! unzip train.zip

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [50]:
import csv

INPUT_DATA = 'train.csv'
#INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

{'BodyMarkdown': "I'm new to C#, and I want to use a trackbar for the forms opacity\r\nThis is my code\r\n\r\n    decimal trans = trackBar1.Value / 5000\r\n    this.Opacity = trans\r\n\r\nWhen I try to build it, I get this error\r\n\r\n**Cannot implicitly convert type 'decimal' to 'double**\r\n\r\nI tried making trans a double, but then the control doesn't work. This code worked fine for me in VB.NET. Any suggestions?",
 'OpenStatus': 'open',
 'OwnerCreationDate': '07/31/2008 21:33:24',
 'OwnerUndeletedAnswerCountAtPostTime': '0',
 'OwnerUserId': '8',
 'PostClosedDate': '',
 'PostCreationDate': '07/31/2008 21:42:52',
 'PostId': '4',
 'ReputationAtPostCreation': '1',
 'Tag1': 'c#',
 'Tag2': '',
 'Tag3': '',
 'Tag4': '',
 'Tag5': '',
 'Title': 'Decimal vs Double?'}

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [2]:
STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, train_output='train', test_output='test', ytest_output='ytest'):
    reader = csv.DictReader(open(INPUT_DATA))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in reader:
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = '%s %s\n' % (label, features)
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [3]:
import re

def extract_title(row):
    title = row['Title']
    # YOUR CODE HERE
    return ''.join([character for character in title.lower() if (character.isalpha() or character == ' ')])

In [ ]:
data2vw(lambda row: '| %s' % extract_title(row))

In [ ]:
! head -n 5 train

Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвав `vw --help`.

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model

Применим модель к тестовой выборке и сохраним предсказания.

In [ ]:
! vw -i model -t test -r pred

In [ ]:
! head -n 3 pred

Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [4]:
def get_scores(ytest_input='ytest', pred_input='pred'):
    import numpy as np
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')
    
    for label, pred in zip(reader_ytest, reader_pred):
        # YOUR CODE HERE
        scores = np.array([
            float(item[item.find(':') + 1:])
            for item in pred.split(' ')
        ])
        label = int(label)
        probabilities = np.exp(scores) / np.sum(np.exp(scores))
        loss += np.log(probabilities[label - 1])
        if ((np.argmax(probabilities) + 1) != label):
            error += 1
        n += 1

    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [ ]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [ ]:
# YOUR CODE HERE
# Добавляем n-граммы разных длин
! vw -d train --loss_function logistic --oaa 5 -f model --ngram 2 --ngram 3 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

In [ ]:
# Добавляем n-граммы и n-skips разных длин
! vw -d train --loss_function logistic --oaa 5 -f model --skips 3 --ngram 4 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

In [ ]:
# Добавляем суффиксы
! vw -d train --loss_function logistic --oaa 5 -f model --affix +2 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [ ]:
# YOUR CODE HERE
#  Теперь запустим больше проходов по выборке и увеличим размер хеша.
! vw -d train --loss_function logistic --oaa 5 -f model --skips 3 --ngram 4 --quiet --passes 5 --cache_file cache_file -b 20
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Добавление фичей в основном только ухудшает качество. Лишь добавление суффиксов небольшой длины увеличило его. Увеличение числапроходов и размера хеша уменьшает качество еще сильнее.

Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [5]:
# Вернем лучшую модель, которая была
! vw -d train --loss_function logistic --oaa 5 -f model --affix +2 --quiet
! vw -i model -t test -r pred --quiet

In [6]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Version 8.4.0
Id 
Min label:-50
Max label:50
bits:18
lda:0
0 ngram:
0 skip:
options: --affix +2 --oaa 5
Checksum: 32408175
:0
Constant:142048:-2.02494
Constant[1]:142049:-1.40876
Constant[2]:142050:-1.60224
Constant[3]:142051:1.32718
Constant[4]:142052:-2.2639
a^#visualstudio:174888:-0.0938351
a^#visualstudio[1]:174889:-0.0834189
a^#visualstudio[2]:174890:-0.0914689
a^#visualstudio[3]:174891:0.0811847
a^#visualstudio[4]:174892:-0.0951813
a^--call:206552:-0.023634
a^--call[1]:206553:-0.0270316
a^--call[2]:206554:-0.0274786
a^--call[3]:206555:0.0262607
a^--call[4]:206556:-0.0200461
a^.a:39888:-0.0719136
a^.a[1]:39889:-0.0498216
a^.a[2]:39890:-0.069618
a^.a[3]:39891:0.0475522


Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

In [7]:
# YOUR CODE HERE
import re
from heapq import heappop, heappush

with open('model.readable') as file:
    counter = 0
    label_regular = re.compile(r'\[\d\]')
    feature_regular = re.compile(r'^(.+):\d+:')
    weight_regular = re.compile(r':\d+:(.+)$')
    heaps = {i: [] for i in STATUS_DICT.values()}
    heap_size = 10
    for line in file:
        if counter > 10:
            label = label_regular.findall(line)
            if len(label) != 0:
                label = int(label[0][1:-1])
            else:
                label = None
            feature = feature_regular.findall(line)[0]
            if feature[-1] == ']':
                feature = feature[:-3]
            weight = abs(float(weight_regular.findall(line)[0]))

            if label not in heaps:
                continue
            if label is not None:
                if ((weight, feature) not in heaps[label]):
                    heappush(heaps[label], (weight, feature))
                    if len(heaps[label]) > heap_size:
                        heappop(heaps[label])
                else:
                    print('!!!')
        counter += 1

Посмотрим на кучи для некоторых статусов:

In [8]:
heaps[1]

[(0.976418, 'a^global-asax'),
 (0.976418, 'd^nID,'),
 (0.988981, 'b^2g'),
 (0.988981, 'c^m/D'),
 (0.976418, 't^create'),
 (0.988981, 't^in'),
 (1.40876, 'd^l-Sh'),
 (1.40876, 'Constant'),
 (1.40876, 'd^imel'),
 (1.40876, 'd^way/')]

In [9]:
heaps[2]

[(1.07207, 't^prompt'),
 (1.1132, 'c^hda'),
 (1.1132, 't^possible'),
 (1.35952, 't^using'),
 (1.1132, 't^castleactiverecord'),
 (1.60224, 'd^imel'),
 (1.35952, 'c^XX.'),
 (1.60224, 'Constant'),
 (1.60224, 'd^l-Sh'),
 (1.60224, 'd^way/')]

In [10]:
heaps[4]

[(0.82407, 'd^BMS?'),
 (0.82407, 't^registertypelib'),
 (0.830748, 'c^m/D'),
 (0.830748, 'b^2g'),
 (0.829659, 't^a'),
 (2.2639, 'd^way/'),
 (0.830748, 't^in'),
 (2.2639, 'Constant'),
 (2.2639, 'd^l-Sh'),
 (2.2639, 'd^imel')]

Интересно, что кое-где суффиксы являются сильно значимыми признаками. Таким образом и следует выбирать, какие фичи добавлять.

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Приветствуется добавление информации о пользователе из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [11]:
from collections import Counter
import re


def extract_ngram_body(row, ngram=3):
    # YOUR CODE HERE
    body_text = re.sub('\|', 'ы', row['BodyMarkdown'])
    body_text = re.sub('\:', 'я', body_text)
    body_text = re.sub('\n+', '. ', body_text)
    def ngram_from_word(counter, word, ngram=2):
        for index in range(1, len(word) - ngram + 1):
            counter[word[index:(index + ngram)]] += 1
    ngrams_dict = Counter()
    for word in body_text.split(' '):
        ngram_from_word(ngrams_dict, word, ngram)
    return ' '.join([key + ':' + str(ngrams_dict[key]) for key in ngrams_dict])


def extract_tags(row):
    # YOUR CODE HERE
    return ' '.join(row['Tag' + str(i)] for i in range(1, 6))

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [12]:
extractors_list = [
    ('t', extract_title),
    ('b', extract_ngram_body),
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
        # YOUR CODE HERE
        return ' '.join(['|' + name + ' ' + extractor(row) for name, extractor in extractors_list])
    return feature_extractor

In [ ]:
data2vw(make_feature_extractor(extractors_list))
! head -n 1 train

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. Совсем необязательно перебирать все параметры по сетке и добиваться оптимального качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.

Выберите не менее трех параметров. Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений "руками", а можете воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Какие параметры повлияли на улучшение качества сильнее всего?

In [ ]:
# YOUR CODE HERE

Добавим в фичи n-граммы разных длин.

In [13]:
# Добавим n-граммы разной длины
extractors_list = [
    ('t', extract_title),
    ('b', lambda row: extract_ngram_body(row, 2)),
    ('c', lambda row: extract_ngram_body(row, 3)),
    ('d', lambda row: extract_ngram_body(row, 4)),
    ('a', extract_tags)
] # (namespace, extractor)

data2vw(make_feature_extractor(extractors_list))

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Добавлене таких фичей существенно качество не улучшило. Но небольшой прирост все же дало.

Попробуем l1 и l2 регуляризацию

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l2 0.000001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l1 0.000001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Как видим, l2 регуляризация дала значителное улучшение, а l1 оказалась бесполезной. Лучше брать l2 регуляризацию.

Теперь попробуем улучшить качество, поменяв различный параметры оптимизации.

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --sgd --quiet --passes 1 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --sgd --quiet --passes 5 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --sgd --quiet --passes 10 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Интересно, что для sgd увеличение числа проходов все же положительно складывается на качестве.

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --adaptive --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

Однако в целом изменение режима оптимизации приводит к значительно худшим результатам.

Попробуем уменьшаять learning rate между проходами.

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l2 0.000001 --passes 3 --cache_file cache_file --decay_learning_rate 0.05
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores()) 

Прироста достигнуть не получилось. Попробуем добавить нелинейные признаки несколькими способами.

In [14]:
# Квадратичные признаки по body объекта
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l1 0.000001 -q bb
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 1.60944
accuracy = 0.00904


In [15]:
# Перемножаем признаки из body и title
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l1 0.000001 -q tb
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13167
accuracy = 0.97927


Добавление квадратичных признаков привобит к очень плохой обучаемости. А пот скрестное перемножение сказывается на качестве неплохо.

Займемся добавлением своих фичей. Добавим даты создания поста и дату регистрации пользователя.

In [67]:
def extract_dates(row):
    import datetime
    def get_date_time(string):
        date = [int(x) for x in string.split()[0].split('/')]
        time = [int(x) for x in string.split()[1].split(':')]
        date = datetime.datetime(date[2], date[0], date[1])
        time = datetime.datetime(1970, 1, 1, time[0], time[1], time[2])
        date = str((date - datetime.datetime(1970, 1, 1)).total_seconds())
        time = str((time - datetime.datetime(1970, 1, 1)).total_seconds())
        return date, time

    result = ''
    try:
        owner_creation_date_time = get_date_time(row['OwnerCreationDate'])
        result += owner_creation_date_time[0] + ' ' + owner_creation_date_time[1]
    except ValueError:
        pass
    try:
        post_creation_date_time = get_date_time(row['PostCreationDate'])
        result += post_creation_date_time[0] + ' ' + post_creation_date_time[1]
    except ValueError:
        pass
    return result

In [68]:
extractors_list = [
    ('t', extract_title),
    ('b', extract_ngram_body),
    ('a', extract_tags),
    ('d', extract_dates)
] # (namespace, extractor)

In [69]:
data2vw(make_feature_extractor(extractors_list))

In [70]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l1 0.000001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14416
accuracy = 0.97927


Новые фичи прироста в качестве не дали. Но при этом оно сильно не упало. Это означает, что при некоторых других параметрах (например других параметрах оптимизатора), такие фичи могут хорошо повлиять на качество.

В целом все ясно, самая главная команда: vw -h